# Welcome to Modal notebooks!

Write Python code and collaborate in real time. Your code runs in Modal's
**serverless cloud**, and anyone in the same workspace can join.

This notebook comes with some common Python libraries installed. Run
cells with `Shift+Enter`.

In [1]:
import torch

if torch.cuda.is_available():
    print("CUDA (NVIDIA GPU) is available.")
    print(f"Number of CUDA devices: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"  Device {i} Name: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA (NVIDIA GPU) is not available.")

CUDA (NVIDIA GPU) is not available.


In [2]:
import os

target_dir = './GenImage'
os.makedirs(target_dir, exist_ok=True)
print(f"Created or found directory: {target_dir}")
os.chdir(target_dir)
print(f"Current directory: {os.getcwd()}")

Created or found directory: ./GenImage
Current directory: /root/GenImage


In [3]:
print(os.listdir())

['WeatherDiffusion']


In [4]:
import subprocess, os, sys

repo_url_ssh = "https://github.com/Hadayxinchao/WeatherDiffusion.git"
repo_url_https = "https://github.com/Hadayxinchao/WeatherDiffusion.git"
repo_dir = "WeatherDiffusion"

if os.path.exists(repo_dir):
    print(f"Repo already exists at {repo_dir}, skipping clone.")
else:
    try:
        print(f"Cloning via SSH: {repo_url_ssh}")
        subprocess.check_call(["git", "clone", repo_url_ssh])
    except Exception as e:
        print(f"SSH clone failed ({e}); falling back to HTTPS...")
        subprocess.check_call(["git", "clone", repo_url_https])

os.chdir(repo_dir)
print("Now in repo:", os.getcwd())
print("Git remotes:")
subprocess.check_call(["git", "remote", "-v"])


Repo already exists at WeatherDiffusion, skipping clone.
Now in repo: /root/GenImage/WeatherDiffusion
Git remotes:
origin	https://github.com/Hadayxinchao/WeatherDiffusion.git (fetch)
origin	https://github.com/Hadayxinchao/WeatherDiffusion.git (push)


0

In [5]:
print(os.listdir())

['.git', '.github', '.gitignore', 'LICENSE', 'README.md', 'calculate_psnr_ssim.py', 'configs', 'datasets', 'eval_diffusion.py', 'models', 'test_ohaze.py', 'train_diffusion.py', 'unet_finetune.ipynb', 'utils', 'weatherdiff.ipynb', 'data']


# Data prepare

In [6]:
# Install gdown for downloading from Google Drive
!pip install -q gdown

import gdown
import zipfile
import os
import shutil

# --- CẤU HÌNH ---
FILE_ID = '1tPXAPoyQVHwriAfkEcOn7Q9yop4Z7ev4' 

# 2. Đường dẫn tải về và thư mục đích
DOWNLOAD_OUTPUT = 'dataset.zip'
EXTRACT_DIR = './data/custom_haze' # Thư mục chứa ảnh sau khi giải nén

# --- XỬ LÝ ---
# Tạo thư mục nếu chưa có
if not os.path.exists(EXTRACT_DIR):
    os.makedirs(EXTRACT_DIR)

# Download file ZIP
print(f"⬇️ Đang tải file zip (ID: {FILE_ID})...")
try:
    # Dùng fuzzy=True để gdown tự trích xuất ID nếu bạn lỡ paste cả link
    gdown.download(id=FILE_ID, output=DOWNLOAD_OUTPUT, quiet=False, fuzzy=True)
    
    # Kiểm tra file tải về
    if os.path.exists(DOWNLOAD_OUTPUT):
        print("📦 Đang giải nén dữ liệu...")
        try:
            with zipfile.ZipFile(DOWNLOAD_OUTPUT, 'r') as zip_ref:
                zip_ref.extractall(EXTRACT_DIR)
            print(f"✅ Đã giải nén thành công vào: {EXTRACT_DIR}")
            
            # Xóa file zip cho nhẹ máy
            os.remove(DOWNLOAD_OUTPUT)
            
            # Kiểm tra nhanh số lượng file
            num_files = sum([len(files) for r, d, files in os.walk(EXTRACT_DIR)])
            print(f"📊 Tổng số file ảnh tìm thấy: {num_files}")
            
        except zipfile.BadZipFile:
            print("❌ Lỗi: File tải về không phải là file zip hợp lệ. Hãy kiểm tra lại link Drive.")
    else:
        print("❌ Lỗi: Không tải được file. Kiểm tra lại ID và quyền truy cập (Share: Anyone with link).")

except Exception as e:
    print(f"❌ Có lỗi xảy ra: {e}")


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
⬇️ Đang tải file zip (ID: 1tPXAPoyQVHwriAfkEcOn7Q9yop4Z7ev4)...


Downloading...
From (original): https://drive.google.com/uc?id=1tPXAPoyQVHwriAfkEcOn7Q9yop4Z7ev4
From (redirected): https://drive.google.com/uc?id=1tPXAPoyQVHwriAfkEcOn7Q9yop4Z7ev4&confirm=t&uuid=b195faeb-8efd-4bde-8c91-72485235d48b
To: /root/GenImage/WeatherDiffusion/dataset.zip
100%|███████████████████████████████████████████████████████████████████████| 377M/377M [00:06<00:00, 60.7MB/s]


📦 Đang giải nén dữ liệu...
✅ Đã giải nén thành công vào: ./data/custom_haze
📊 Tổng số file ảnh tìm thấy: 5482


In [7]:
import os

# Định nghĩa lại biến data_dir cho khớp với thư mục giải nén
data_dir = './data/custom_haze' 

# Kiểm tra cấu trúc thư mục
if os.path.exists(data_dir):
    print(f"Checking structure for: {data_dir}")
    print("Dataset structure:")
    for root, dirs, files in os.walk(data_dir):
        level = root.replace(data_dir, '').count(os.sep)
        indent = ' ' * 2 * level
        print(f'{indent}{os.path.basename(root)}/')
        subindent = ' ' * 2 * (level + 1)
        for file in files[:5]:  # Only show first 5 files
            print(f'{subindent}{file}')
        if len(files) > 5:
            print(f'{subindent}... and {len(files) - 5} more files')
else:
    print(f"❌ Thư mục {data_dir} chưa tồn tại. Hãy chạy cell tải dữ liệu (Data prepare) ở trên trước!")

Checking structure for: ./data/custom_haze
Dataset structure:
custom_haze/
  train.txt
  test.txt
  output/
    test/
      results_prediction/
        161.jpg
        170.jpg
        173.jpg
        176.jpg
        153.jpg
        ... and 445 more files
      test_haze/
        161.jpg
        170.jpg
        176.jpg
        164.jpg
        151.jpg
        ... and 445 more files
      test_origin/
        96.jpg
        99.jpg
        82.jpg
        95.jpg
        88.jpg
        ... and 445 more files
    train/
      train_haze/
        1612.jpg
        1582.jpg
        1733.jpg
        1714.jpg
        1619.jpg
        ... and 2060 more files
      train_origin/
        994.jpg
        986.jpg
        998.jpg
        985.jpg
        990.jpg
        ... and 2060 more files


In [8]:
# Di chuyển vào thư mục output (nơi chứa train và test folders)
output_dir = os.path.join(data_dir, 'output')
print(f"Output directory: {output_dir}")
print(f"Contents: {os.listdir(output_dir) if os.path.exists(output_dir) else 'Directory not found'}")

Output directory: ./data/custom_haze/output
Contents: ['test', 'train']


In [9]:
import os
import glob

# Tạo file danh sách cho training set
# Với custom dataset, train_haze chứa ảnh haze nhưng không có GT riêng
# Chúng ta cần tạo paired list giữa haze và origin images

train_haze_dir = os.path.join(output_dir, 'train', 'train_haze')
test_haze_dir = os.path.join(output_dir, 'test', 'test_haze')
test_origin_dir = os.path.join(output_dir, 'test', 'test_origin')

# Get all training haze images
train_haze_images = sorted(glob.glob(os.path.join(train_haze_dir, '*.*')))
print(f"Found {len(train_haze_images)} training haze images")

# Get all test images
test_haze_images = sorted(glob.glob(os.path.join(test_haze_dir, '*.*')))
test_origin_images = sorted(glob.glob(os.path.join(test_origin_dir, '*.*')))
print(f"Found {len(test_haze_images)} test haze images")
print(f"Found {len(test_origin_images)} test origin images")

# Create train.txt - for self-supervised training (haze image as both input and target)
# Or you can use haze as input and haze as output for denoising
train_list_path = os.path.join(data_dir, 'train.txt')
with open(train_list_path, 'w') as f:
    for haze_img in train_haze_images:
        # Format: input_path gt_path
        # Nếu không có GT riêng, dùng chính ảnh haze làm target (self-supervised)
        # Hoặc bạn có thể điều chỉnh nếu có GT riêng
        f.write(f"{haze_img} {haze_img}\n")
print(f"Created {train_list_path}")

# Create test.txt - paired haze and origin images
test_list_path = os.path.join(data_dir, 'test.txt')
with open(test_list_path, 'w') as f:
    for haze_img, origin_img in zip(test_haze_images, test_origin_images):
        f.write(f"{haze_img} {origin_img}\n")
print(f"Created {test_list_path}")

# Hiển thị vài dòng đầu tiên
print("\nFirst 3 lines of train.txt:")
with open(train_list_path, 'r') as f:
    for i, line in enumerate(f):
        if i >= 3:
            break
        print(f"  {line.strip()}")

print("\nFirst 3 lines of test.txt:")
with open(test_list_path, 'r') as f:
    for i, line in enumerate(f):
        if i >= 3:
            break
        print(f"  {line.strip()}")

Found 2065 training haze images
Found 450 test haze images
Found 450 test origin images
Created ./data/custom_haze/train.txt
Created ./data/custom_haze/test.txt

First 3 lines of train.txt:
  ./data/custom_haze/output/train/train_haze/1000.jpg ./data/custom_haze/output/train/train_haze/1000.jpg
  ./data/custom_haze/output/train/train_haze/1001.jpg ./data/custom_haze/output/train/train_haze/1001.jpg
  ./data/custom_haze/output/train/train_haze/1002.jpg ./data/custom_haze/output/train/train_haze/1002.jpg

First 3 lines of test.txt:
  ./data/custom_haze/output/test/test_haze/10.jpg ./data/custom_haze/output/test/test_origin/10.jpg
  ./data/custom_haze/output/test/test_haze/100.jpg ./data/custom_haze/output/test/test_origin/100.jpg
  ./data/custom_haze/output/test/test_haze/102.jpg ./data/custom_haze/output/test/test_origin/102.jpg


In [10]:
# Kiểm tra cấu trúc thư mục cuối cùng
print("Current directory:", os.getcwd())
print("\nData directory structure:")
print(f"data_dir in config: {os.path.join(os.getcwd(), 'data', 'custom_haze')}")
print(f"Exists: {os.path.exists(os.path.join(os.getcwd(), 'data', 'custom_haze'))}")

# Kiểm tra file lists
train_txt = os.path.join(os.getcwd(), 'data', 'custom_haze', 'train.txt')
test_txt = os.path.join(os.getcwd(), 'data', 'custom_haze', 'test.txt')
print(f"\ntrain.txt exists: {os.path.exists(train_txt)}")
print(f"test.txt exists: {os.path.exists(test_txt)}")

Current directory: /root/GenImage/WeatherDiffusion

Data directory structure:
data_dir in config: /root/GenImage/WeatherDiffusion/data/custom_haze
Exists: True

train.txt exists: True
test.txt exists: True


# Train

In [ ]:
import subprocess
import sys

process = subprocess.Popen([
    'python', 'train_diffusion.py',
    '--config', 'mydataset.yml',
    '--resume', 'WeatherDiff64.pth.tar'
], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

try:
    for line in process.stdout:
        print(line, end='')
        sys.stdout.flush()
    process.wait()
    if process.returncode != 0:
        print(f"Process exited with code {process.returncode}")
except Exception as e:
    print(f"Error: {e}")

Using device: cuda
=> using dataset 'MyDataset'
Data Loaded!
=> creating denoising-diffusion model...
Found 40 images in ./haze_data/train/hazy
Found 5 images in ./haze_data/test/hazy
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    decoupled_weight_decay: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 1e-05
    maximize: False
    weight_decay: 0.001
)
Current learning rate before training: 1e-05
epoch:  0
epoch:  1
epoch:  2
epoch:  3
step: 10, loss: 10449.59375, data time: 3.966639518737793
epoch:  4
epoch:  5
epoch:  6
step: 20, loss: 7910.8818359375, data time: 2.0040950775146484
epoch:  7
epoch:  8
epoch:  9
step: 30, loss: 6066.8623046875, data time: 1.216320514678955
epoch:  10
epoch:  11
epoch:  12
epoch:  13
step: 40, loss: 4578.3701171875, data time: 3.9093542098999023
epoch:  14
epoch:  15
epoch:  16
step: 50, loss: 3654.35595703125, data time: 1.8904286623001099
epoch:  17
epoch:  18
epo

In [12]:
%%writefile datasets/__init__.py
from .snow100k import Snow100K
from .raindrop import RainDrop
from .outdoorrain import OutdoorRain
from .ohaze import OHaze
from .custom_haze import CustomHaze
from .allweather import AllWeather

def get_dataset(args, config):
    # Lấy tên dataset từ file config
    dataset_name = config.data.dataset
    
    if dataset_name == 'Snow100K':
        return Snow100K(args, config)
    elif dataset_name == 'RainDrop':
        return RainDrop(args, config)
    elif dataset_name == 'OutdoorRain':
        return OutdoorRain(args, config)
    elif dataset_name == 'OHaze':
        return OHaze(args, config)
    elif dataset_name == 'AllWeather':
        return AllWeather(args, config)
    
    # --- SỬA Ở ĐÂY ---
    # Thay vì so sánh == 'CustomHaze', ta dùng startswith
    # Để chấp nhận cả 'CustomHaze', 'CustomHaze_case1', 'CustomHaze_case2'...
    elif dataset_name.startswith('CustomHaze'):
        return CustomHaze(args, config)
    
    else:
        raise KeyError(f"Dataset không hợp lệ: {dataset_name}. Hãy kiểm tra lại configs.")

Overwriting datasets/__init__.py


In [ ]:
# Xóa các file config cũ để tránh conflict
import os
import glob

old_configs = glob.glob("configs/case*.yml")
for f in old_configs:
    try:
        os.remove(f)
        print(f"🗑️ Đã xóa file config cũ: {f}")
    except:
        pass

print("✅ Sẵn sàng tạo config mới!")

In [14]:
import os
import yaml
import subprocess
import sys
import modal
import shutil

# --- CẤU HÌNH ---
BASE_CONFIG_PATH = "configs/custom_haze.yml"
VOL_NAME = "weather-diffusion-vol"

# Tên file model gốc bạn muốn dùng để finetune (CẦN CÓ SẴN trong folder ./ckpts)
# Nếu bạn chưa có, hãy đảm bảo đã upload file WeatherDiff64.pth.tar vào ./ckpts 
# hoặc code sẽ tự tìm trong Modal Volume
BASE_MODEL_NAME = "WeatherDiff64.pth.tar" 

# Định nghĩa 5 thí nghiệm
experiments = {
    "case1_baseline": {
        "desc": "Config Goc (No changes)",
        "changes": {} 
    },
    "case2_highWD": {
        "desc": "Tang Weight Decay -> 0.01",
        "changes": {"optim": {"weight_decay": 0.01}}
    },
    "case3_lowLR": {
        "desc": "Giam LR -> 5e-6",
        "changes": {"optim": {"lr": 0.000005}}
    },
    "case4_dropout": {
        "desc": "Tang Dropout -> 0.2",
        "changes": {"model": {"dropout": 0.2}}
    },
    "case5_combined": {
        "desc": "Ket hop tat ca: WD=0.01, LR=5e-6, Dropout=0.2",
        "changes": {
            "optim": {"weight_decay": 0.01, "lr": 0.000005},
            "model": {"dropout": 0.2}
        }
    }
}

# Hàm helper update config
def update_nested_dict(d, u):
    for k, v in u.items():
        if isinstance(v, dict):
            d[k] = update_nested_dict(d.get(k, {}), v)
        else:
            d[k] = v
    return d

# Kết nối Volume
vol = modal.Volume.from_name(VOL_NAME, create_if_missing=True)

# Kiểm tra base model local
if not os.path.exists(f"./ckpts/{BASE_MODEL_NAME}"):
    print(f"⚠️ CẢNH BÁO: Không tìm thấy './ckpts/{BASE_MODEL_NAME}'.")
    print("   Code sẽ cố gắng tìm trên Volume hoặc train từ đầu (Random Init).")
    try:
        vol.read_file(f"checkpoints/{BASE_MODEL_NAME}", f"./ckpts/{BASE_MODEL_NAME}")
        print("   ✅ Đã tải Base Model từ Volume về.")
    except:
        pass

# --- VÒNG LẶP CHÍNH ---
for case_name, exp_data in experiments.items():
    print(f"\n{'='*60}")
    print(f"🚀 RUNNING EXPERIMENT: {case_name}")
    print(f"📝 {exp_data['desc']}")
    print(f"{'='*60}")

    # 1. Tạo Config
    with open(BASE_CONFIG_PATH, 'r') as f:
        cfg = yaml.safe_load(f)
    
    cfg = update_nested_dict(cfg, exp_data["changes"])
    # KHÔNG đổi dataset name, giữ nguyên "CustomHaze"
    config_file = f"configs/{case_name}.yml"
    with open(config_file, 'w') as f:
        yaml.dump(cfg, f)
    
    # 2. Chuẩn bị file Resume/Pretrained
    # File checkpoint sẽ có tên: CustomHaze_ddpm.pth.tar (mặc định)
    # Để tránh đè lên nhau, ta sẽ đổi tên sau khi train xong
    default_ckpt_name = "CustomHaze_ddpm.pth.tar"
    target_ckpt_name = f"{case_name}_model.pth.tar"
    local_ckpt_path = f"./ckpts/{default_ckpt_name}"
    final_ckpt_path = f"./ckpts/{target_ckpt_name}"
    
    resume_path = ""
    
    # Check xem case này đã từng chạy dở trên Volume chưa?
    try:
        vol_files = [e.path for e in vol.listdir(f"experiments/{case_name}")]
        if target_ckpt_name in [os.path.basename(p) for p in vol_files]:
            print("🔄 Phát hiện checkpoint cũ trên Volume. Đang tải về để RESUME...")
            vol.read_file(f"experiments/{case_name}/{target_ckpt_name}", local_ckpt_path)
            resume_path = local_ckpt_path
    except:
        pass 

    # Nếu chưa có file resume, dùng Base Model để bắt đầu Finetune
    if not resume_path and os.path.exists(f"./ckpts/{BASE_MODEL_NAME}"):
        print("🆕 Chưa có checkpoint riêng. Copy Base Model để bắt đầu FINETUNE...")
        shutil.copy(f"./ckpts/{BASE_MODEL_NAME}", local_ckpt_path)
        resume_path = local_ckpt_path
    
    # 3. Chạy Training
    cmd = [
        'python', 'train_diffusion.py',
        '--config', f'{case_name}.yml',
        '--sampling_timesteps', '25',
        '--image_folder', f'results/{case_name}_patches/'
    ]
    if resume_path:
        cmd.extend(['--resume', resume_path])

    print(f"▶️ Executing: {' '.join(cmd)}")
    
    try:
        process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
        for line in process.stdout:
            # Chỉ in các dòng quan trọng để log đỡ dài
            if any(k in line for k in ["epoch:", "step:", "Error", "Resume", "loaded checkpoint"]):
                print(line.strip())
        process.wait()
        
        if process.returncode != 0:
            print(f"❌ Training failed with code {process.returncode}")
            continue
            
    except KeyboardInterrupt:
        print("\n🛑 Dừng bởi người dùng.")
        break # Dừng toàn bộ nếu user bấm stop
    except Exception as e:
        print(f"❌ Lỗi: {e}")
        continue


    # 4. Đổi tên checkpoint để tránh đè lên nhau giữa các experiment
    if os.path.exists(local_ckpt_path):
        shutil.move(local_ckpt_path, final_ckpt_path)
        print(f"📝 Đã đổi tên checkpoint: {default_ckpt_name} -> {target_ckpt_name}")
    
    # 5. Lưu kết quả lên Volume
    print(f"\n💾 Đang lưu model {case_name} lên Volume...")
    if os.path.exists(final_ckpt_path):
        remote_path = f"experiments/{case_name}/{target_ckpt_name}"
        with vol.batch_upload() as batch:
            batch.put_file(final_ckpt_path, remote_path)
        print(f"✅ Đã lưu: {remote_path}")
    else:
        print("⚠️ Không tìm thấy file model output để lưu.")

print("\n🎉 HOÀN TẤT TOÀN BỘ 5 THÍ NGHIỆM!")

⚠️ CẢNH BÁO: Không tìm thấy './ckpts/WeatherDiff64.pth.tar'.
   Code sẽ cố gắng tìm trên Volume hoặc train từ đầu (Random Init).
   ✅ Đã tải Base Model từ Volume về.

🚀 RUNNING EXPERIMENT: case1_baseline
📝 Config Goc (No changes)
▶️ Executing: python train_diffusion.py --config case1_baseline.yml --sampling_timesteps 25 --image_folder results/case1_baseline_patches/
KeyError: 'CustomHaze_case1_baseline'

💾 Đang lưu model case1_baseline lên Volume...
⚠️ Không tìm thấy file model output để lưu.

🚀 RUNNING EXPERIMENT: case2_highWD
📝 Tang Weight Decay -> 0.01
▶️ Executing: python train_diffusion.py --config case2_highWD.yml --sampling_timesteps 25 --image_folder results/case2_highWD_patches/
KeyError: 'CustomHaze_case2_highWD'

💾 Đang lưu model case2_highWD lên Volume...
⚠️ Không tìm thấy file model output để lưu.

🚀 RUNNING EXPERIMENT: case3_lowLR
📝 Giam LR -> 5e-6
▶️ Executing: python train_diffusion.py --config case3_lowLR.yml --sampling_timesteps 25 --image_folder results/case3_lowLR_p